In [29]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm   #barres de chargement pour les tâches longues (tracking)
import numpy as np
from google.colab import drive #acces au Drive
import os

from scipy.optimize import minimize
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
drive.mount("/content/drive", force_remount = True)

chemin = 'drive/MyDrive/M2 SID/Last try/Projet non-alternant/' #perso

Mounted at /content/drive


In [ ]:
df_audio = pd.read_csv(chemin + 'Audio/CNN/' + 'test_predictions_audio.csv')
df_video = pd.read_csv(chemin + 'Vidéo/test/' + 'test_predictions_video.csv', sep = ";")
df_texte = pd.read_csv(chemin + 'Texte/' + 'test_predictions_texte.csv')

In [ ]:
df_audio

,music,people,gaming,sports/actions,news/events/politics,education,tv shows,movie/comedy,animation,vehicles/autos,...,animals/pets,kids/family,documentary,food/drink,cooking,beauty/fashion,advertisement,true_label,pred_label,video_name
0,0.122012,0.060279,0.049795,0.164159,0.016570,0.007865,0.021536,0.081991,0.078305,0.068031,...,0.079977,0.025666,0.010088,0.024786,0.015788,0.048857,0.018784,howto,sports/actions,video7010.mp4
1,0.165756,0.069629,0.034980,0.126276,0.019382,0.007584,0.028013,0.067135,0.053296,0.042681,...,0.068610,0.056665,0.014773,0.035628,0.026421,0.078080,0.019085,vehicles/autos,music,video7011.mp4
2,0.000028,0.000302,0.001481,0.039028,0.268311,0.240718,0.003380,0.025185,0.000058,0.091814,...,0.019068,0.000373,0.007926,0.045749,0.004474,0.009204,0.000009,education,news/events/politics,video7012.mp4
3,0.011108,0.015587,0.036116,0.079722,0.094561,0.058726,0.041381,0.112984,0.016742,0.104521,...,0.057745,0.022845,0.039621,0.062474,0.027022,0.026943,0.012856,kids/family,movie/comedy,video7014.mp4
4,0.066026,0.035630,0.063938,0.056524,0.023757,0.011963,0.034199,0.090972,0.049223,0.043127,...,0.086812,0.071574,0.015173,0.073850,0.057924,0.075425,0.026076,movie/comedy,movie/comedy,video7015.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630,0.050975,0.019678,0.095705,0.090863,0.011271,0.005329,0.021705,0.217900,0.061424,0.063285,...,0.084473,0.022022,0.007463,0.036232,0.025552,0.049768,0.030339,kids/family,movie/comedy,video9994.mp4
2631,0.075570,0.046304,0.055976,0.068030,0.033213,0.021810,0.036042,0.071615,0.060079,0.057287,...,0.066678,0.067271,0.025772,0.052157,0.036975,0.062137,0.034683,tv shows,music,video9995.mp4
2632,0.001201,0.003548,0.006736,0.011802,0.022925,0.010727,0.032947,0.053582,0.001570,0.006223,...,0.067286,0.008479,0.004019,0.381411,0.178038,0.108396,0.000538,howto,food/drink,video9997.mp4
2633,0.002636,0.006907,0.012906,0.098897,0.184697,0.089170,0.015907,0.065547,0.003004,0.157320,...,0.048584,0.006350,0.025618,0.052211,0.010946,0.029644,0.001289,news/events/politics,news/events/politics,video9998.mp4


In [ ]:
df_video

,music,people,gaming,sports/actions,news/events/politics,education,tv shows,movie/comedy,animation,vehicles/autos,...,animals/pets,kids/family,documentary,food/drink,cooking,beauty/fashion,advertisement,true_label,pred_label,video_name
0,0.003091,0.007122,0.152244,0.322867,0.047232,0.036117,0.034934,0.016620,0.009588,0.009312,...,0.030467,0.002283,0.017300,0.004121,0.002191,0.002367,0.003703,howto,sports/actions,video7010.mp4
1,0.025044,0.013786,0.053866,0.015797,0.011158,0.127307,0.037126,0.028110,0.200609,0.024461,...,0.026257,0.013450,0.014011,0.015029,0.014377,0.014316,0.096695,vehicles/autos,science/technology,video7011.mp4
2,0.003135,0.004929,0.004361,0.012502,0.048800,0.604785,0.017404,0.026395,0.002520,0.001085,...,0.027168,0.002726,0.016146,0.005779,0.001514,0.002914,0.010614,education,education,video7012.mp4
3,0.029015,0.012676,0.000231,0.000384,0.000514,0.000868,0.004221,0.001516,0.000199,0.000137,...,0.000544,0.941080,0.000528,0.001292,0.000287,0.005624,0.000221,kids/family,kids/family,video7014.mp4
4,0.020960,0.028207,0.003398,0.028647,0.172600,0.016125,0.137146,0.454908,0.001384,0.001592,...,0.008965,0.009906,0.043346,0.009307,0.000545,0.019766,0.021115,movie/comedy,movie/comedy,video7015.mp4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629,0.300976,0.125509,0.002761,0.245893,0.008214,0.016411,0.020635,0.016086,0.002176,0.002082,...,0.017185,0.065144,0.012341,0.016998,0.004732,0.053873,0.004186,kids/family,music,video9994.mp4
2630,0.190216,0.082902,0.011290,0.047844,0.008771,0.004493,0.164121,0.331486,0.006361,0.002433,...,0.008107,0.053317,0.014579,0.005356,0.000684,0.046415,0.012600,tv shows,movie/comedy,video9995.mp4
2631,0.010540,0.014164,0.000557,0.003519,0.000967,0.001380,0.005017,0.004943,0.001997,0.001555,...,0.003108,0.010098,0.001763,0.004202,0.002266,0.875519,0.006828,howto,beauty/fashion,video9997.mp4
2632,0.000638,0.001536,0.000811,0.017958,0.901617,0.008292,0.006816,0.003553,0.000102,0.001144,...,0.000811,0.000695,0.042523,0.001045,0.000107,0.000523,0.000613,news/events/politics,news/events/politics,video9998.mp4


In [ ]:
df_texte

,video_name,music,people,gaming,sports/actions,news/events/politics,education,tv shows,movie/comedy,animation,...,science/technology,animals/pets,kids/family,documentary,food/drink,cooking,beauty/fashion,advertisement,pred_label,true_label
0,video7010.mp4,0.040307,0.012982,0.048386,0.127694,0.008301,0.023873,0.082573,0.011612,0.046930,...,0.090826,0.012164,0.009013,0.002657,0.013492,0.016065,0.010044,0.004115,howto,howto
1,video7011.mp4,0.073949,0.011040,0.024246,0.026991,0.010314,0.021444,0.012452,0.028875,0.147436,...,0.108487,0.030691,0.131629,0.005981,0.006441,0.025955,0.023313,0.112586,animation,vehicles/autos
2,video7012.mp4,0.006165,0.011902,0.003776,0.012464,0.147476,0.463729,0.022750,0.078726,0.009239,...,0.153068,0.019740,0.005304,0.012904,0.010797,0.002265,0.011646,0.004889,education,education
3,video7014.mp4,0.052630,0.007329,0.001993,0.013271,0.006294,0.004192,0.007608,0.012348,0.001236,...,0.004225,0.002672,0.864692,0.001988,0.001474,0.001163,0.010861,0.000685,kids/family,kids/family
4,video7015.mp4,0.254541,0.110199,0.016748,0.049827,0.016769,0.009461,0.043277,0.205617,0.031963,...,0.011041,0.023087,0.103339,0.009480,0.031659,0.009314,0.041742,0.007957,music,movie/comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2630,video9994.mp4,0.029124,0.011241,0.012628,0.026735,0.004229,0.003094,0.032350,0.176731,0.026570,...,0.005912,0.032176,0.518115,0.001950,0.002800,0.011496,0.059359,0.005899,kids/family,kids/family
2631,video9995.mp4,0.163702,0.060998,0.011796,0.029298,0.032422,0.013676,0.061398,0.213751,0.020882,...,0.024463,0.039851,0.028209,0.019435,0.036807,0.015006,0.143916,0.035634,movie/comedy,tv shows
2632,video9997.mp4,0.006107,0.005601,0.000576,0.001471,0.004545,0.001677,0.004917,0.007791,0.001297,...,0.004307,0.004883,0.005632,0.004721,0.007361,0.010684,0.760735,0.002186,beauty/fashion,howto
2633,video9998.mp4,0.003803,0.003604,0.000447,0.014609,0.786115,0.034040,0.028904,0.059662,0.000597,...,0.021645,0.002844,0.001737,0.025980,0.006373,0.000541,0.002692,0.001209,news/events/politics,news/events/politics


In [ ]:
class_cols = [
    "music","people","gaming","sports/actions","news/events/politics",
    "education","tv shows","movie/comedy","animation","vehicles/autos",
    "howto","travel","science/technology","animals/pets","kids/family",
    "documentary","food/drink","cooking","beauty/fashion","advertisement"
]

df_audio_video = pd.merge(df_audio[["video_name"] + class_cols], df_video[["video_name"] + class_cols], on = 'video_name', suffixes = ("_audio", "_video"), how = "inner")
df_audio_video_texte = pd.merge(df_audio_video, df_texte, on = 'video_name', how = "inner")

In [ ]:
df_audio_video_texte

,video_name,music_audio,people_audio,gaming_audio,sports/actions_audio,news/events/politics_audio,education_audio,tv shows_audio,movie/comedy_audio,animation_audio,...,science/technology,animals/pets,kids/family,documentary,food/drink,cooking,beauty/fashion,advertisement,pred_label,true_label
0,video7010.mp4,0.122012,0.060279,0.049795,0.164159,0.016570,0.007865,0.021536,0.081991,0.078305,...,0.090826,0.012164,0.009013,0.002657,0.013492,0.016065,0.010044,0.004115,howto,howto
1,video7011.mp4,0.165756,0.069629,0.034980,0.126276,0.019382,0.007584,0.028013,0.067135,0.053296,...,0.108487,0.030691,0.131629,0.005981,0.006441,0.025955,0.023313,0.112586,animation,vehicles/autos
2,video7012.mp4,0.000028,0.000302,0.001481,0.039028,0.268311,0.240718,0.003380,0.025185,0.000058,...,0.153068,0.019740,0.005304,0.012904,0.010797,0.002265,0.011646,0.004889,education,education
3,video7014.mp4,0.011108,0.015587,0.036116,0.079722,0.094561,0.058726,0.041381,0.112984,0.016742,...,0.004225,0.002672,0.864692,0.001988,0.001474,0.001163,0.010861,0.000685,kids/family,kids/family
4,video7015.mp4,0.066026,0.035630,0.063938,0.056524,0.023757,0.011963,0.034199,0.090972,0.049223,...,0.011041,0.023087,0.103339,0.009480,0.031659,0.009314,0.041742,0.007957,music,movie/comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629,video9994.mp4,0.050975,0.019678,0.095705,0.090863,0.011271,0.005329,0.021705,0.217900,0.061424,...,0.005912,0.032176,0.518115,0.001950,0.002800,0.011496,0.059359,0.005899,kids/family,kids/family
2630,video9995.mp4,0.075570,0.046304,0.055976,0.068030,0.033213,0.021810,0.036042,0.071615,0.060079,...,0.024463,0.039851,0.028209,0.019435,0.036807,0.015006,0.143916,0.035634,movie/comedy,tv shows
2631,video9997.mp4,0.001201,0.003548,0.006736,0.011802,0.022925,0.010727,0.032947,0.053582,0.001570,...,0.004307,0.004883,0.005632,0.004721,0.007361,0.010684,0.760735,0.002186,beauty/fashion,howto
2632,video9998.mp4,0.002636,0.006907,0.012906,0.098897,0.184697,0.089170,0.015907,0.065547,0.003004,...,0.021645,0.002844,0.001737,0.025980,0.006373,0.000541,0.002692,0.001209,news/events/politics,news/events/politics


In [ ]:
df_avg = pd.DataFrame()
df_avg["video_name"] = df_audio_video_texte["video_name"]

# true_label unique (ils doivent être identiques)
df_avg["true_label"] = df_audio_video_texte["true_label"]

coef_audio = 1 / 3
coef_video = 1 / 3
coef_texte = 1 / 3

# moyenne des 3 distributions
for col in class_cols:
  df_avg[col] = (df_audio_video_texte[f"{col}_audio"] * coef_audio + df_audio_video_texte[f"{col}_video"] * coef_video + df_audio_video_texte[col]) * coef_texte

df_avg["pred_label"] = df_avg[class_cols].idxmax(axis = 1)

In [ ]:
df_avg

,video_name,true_label,music,people,gaming,sports/actions,news/events/politics,education,tv shows,movie/comedy,...,travel,science/technology,animals/pets,kids/family,documentary,food/drink,cooking,beauty/fashion,advertisement,pred_label
0,video7010.mp4,howto,0.094777,0.044514,0.049326,0.152004,0.013814,0.013201,0.041882,0.058532,...,0.010825,0.046684,0.057372,0.020115,0.007611,0.021021,0.015880,0.035919,0.013894,howto
1,video7011.mp4,vehicles/autos,0.135154,0.050099,0.031402,0.093181,0.016359,0.012204,0.022826,0.054382,...,0.019226,0.046560,0.055970,0.081653,0.011842,0.025899,0.026266,0.059824,0.050252,music
2,video7012.mp4,education,0.002074,0.004169,0.002246,0.030173,0.228032,0.315055,0.009837,0.043032,...,0.007357,0.198061,0.019292,0.002017,0.009586,0.034098,0.003737,0.010018,0.001636,education
3,video7014.mp4,kids/family,0.024949,0.012834,0.024742,0.057572,0.065139,0.040548,0.030123,0.079439,...,0.026688,0.062527,0.039387,0.303460,0.027077,0.042141,0.018402,0.021582,0.008799,kids/family
4,video7015.mp4,movie/comedy,0.128864,0.060487,0.048208,0.054292,0.021428,0.011129,0.037225,0.129187,...,0.018153,0.020340,0.065571,0.082162,0.013275,0.059786,0.041721,0.064197,0.020036,movie/comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2629,video9994.mp4,kids/family,0.043692,0.016865,0.068013,0.069487,0.008924,0.004584,0.025253,0.204177,...,0.008144,0.023845,0.067041,0.187386,0.005626,0.025088,0.020867,0.052965,0.022192,movie/comedy
2630,video9995.mp4,tv shows,0.104947,0.051202,0.041249,0.055120,0.032949,0.019099,0.044494,0.118994,...,0.028548,0.032497,0.057736,0.054250,0.023659,0.047040,0.029652,0.089396,0.035000,movie/comedy
2631,video9997.mp4,howto,0.002836,0.004232,0.004682,0.008358,0.016798,0.007711,0.023604,0.038318,...,0.013672,0.018160,0.046485,0.007530,0.004253,0.256727,0.122254,0.325842,0.001087,beauty/fashion
2632,video9998.mp4,news/events/politics,0.003025,0.005806,0.008753,0.070801,0.385170,0.070793,0.020240,0.063585,...,0.019409,0.093373,0.033337,0.004812,0.025738,0.036932,0.007477,0.020660,0.001262,news/events/politics


In [ ]:
accuracy = (df_avg["true_label"] == df_avg["pred_label"]).mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.6750189825360668


In [ ]:
dict_accuracy = dict()
coef_possible = np.linspace(.01, .8, 80)
for coef_audio in tqdm(coef_possible) :
  coef_audio = np.round(coef_audio, 2)
  coef_possible_video = np.linspace(.01, .8 - coef_audio, 10 * int(8 - 10 * coef_audio))
  for coef_video in coef_possible_video :
    coef_video = np.round(coef_video, 2)
    coef_texte = np.round(1 - coef_audio - coef_video, 2)
    for col in class_cols:
      df_avg[col] = (df_audio_video_texte[f"{col}_audio"] * coef_audio + df_audio_video_texte[f"{col}_video"] * coef_video + df_audio_video_texte[col]) * coef_texte
      df_avg["pred_label"] = df_avg[class_cols].idxmax(axis = 1)
      accuracy = (df_avg["true_label"] == df_avg["pred_label"]).mean()
      #print(f"coef audio : {coef_audio} ; coef video : {coef_video} ; coef texte : {coef_texte} => Accuracy: {accuracy}")
      dict_accuracy[accuracy] = (coef_audio, coef_video, coef_texte)

  0%|          | 0/80 [00:00<?, ?it/s]

In [33]:
for key in sorted(dict_accuracy.keys(), reverse = True)[:3] :
  print(f"Accuracy: {key} ; coef audio : {dict_accuracy[key][0]} ; coef video : {dict_accuracy[key][1]} ; coef texte : {dict_accuracy[key][2]}")

Accuracy: 0.6921032649962034 ; coef audio : 0.12 ; coef video : 0.16 ; coef texte : 0.72
Accuracy: 0.6917236142748672 ; coef audio : 0.14 ; coef video : 0.16 ; coef texte : 0.7
Accuracy: 0.6913439635535308 ; coef audio : 0.15 ; coef video : 0.16 ; coef texte : 0.69
